In [92]:
import os
import sys

import random

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Dataset preprocessing

In [4]:
vdjdb_raw = pd.read_csv('data/raw/vdjdb_paired.tsv', sep='\t')
iedb_raw = pd.read_csv('data/raw/iedb.csv')
mcpas_raw = pd.read_csv('data/raw/mcpas-tcr.csv', encoding='latin1')

/var/folders/w1/mbcsphtx2xz1xqrls0v3d6r80000gn/T/ipykernel_13121/3174298451.py:2: DtypeWarning: Columns (8,10,11,15,16,17,18,19,20,21,22,23,25,30,31,36,37,44,45,46,47,48,49,50,51,52,54,59,60,65,66) have mixed types. Specify dtype option on import or set low_memory=False.
  iedb_raw = pd.read_csv('data/raw/iedb.csv')
/var/folders/w1/mbcsphtx2xz1xqrls0v3d6r80000gn/T/ipykernel_13121/3174298451.py:3: DtypeWarning: Columns (13,17,24,26,28) have mixed types. Specify dtype option on import or set low_memory=False.
  mcpas_raw = pd.read_csv('data/raw/mcpas-tcr.csv', encoding='latin1')


## VDJdb

In [44]:
vdj_cols = ['complex.id', 'CDR3', 'V', 'J', 'Species', 'MHC A','MHC class', 'Epitope']
print('VDJdb raw:', vdjdb_raw.shape)
vdjdb_a = vdjdb_raw[vdjdb_raw['Gene']=='TRA'][vdj_cols].copy()
vdjdb_b = vdjdb_raw[vdjdb_raw['Gene']=='TRB'][vdj_cols].copy()

vdjdb = pd.merge(vdjdb_a, vdjdb_b, on='complex.id')
print('VDJdb paired:', vdjdb.shape)

assert (vdjdb['Species_x'] == vdjdb['Species_y']).all()
assert (vdjdb['Epitope_x'] == vdjdb['Epitope_y']).all()
assert (vdjdb['MHC A_x'] == vdjdb['MHC A_y']).all()

vdj_cols = ['complex.id', 'CDR3_x', 'V_x', 'J_x', 'CDR3_y', 'V_y', 'J_y', 'Species_y', 'MHC A_y', 'MHC class_y', 'Epitope_y']
vdjdb = vdjdb[vdj_cols].copy()
vdj_cols = ['complex.id', 'cdr3a', 'va', 'ja', 'cdr3b', 'vb', 'jb', 'species', 'mhc', 'mhc_class', 'epitope']
vdjdb.columns = vdj_cols
vdjdb.head()


VDJdb raw: (2502, 17)
VDJdb paired: (1251, 15)


,complex.id,cdr3a,va,ja,cdr3b,vb,jb,species,mhc,mhc_class,epitope
0,15,CAVAGYGGSQGNLIF,TRAV12-2*01,TRAJ42*01,CASSPQGLGTEAFF,TRBV28*01,TRBJ1-1*01,HomoSapiens,HLA-A*02,MHCI,ELAGIGILTV
1,103,CAVTTDSWGKLQF,TRAV12-2*01,TRAJ24*01,CASRPGLAGGRPEQYF,TRBV6-5*01,TRBJ2-7*01,HomoSapiens,HLA-A*02:01:59,MHCI,LLFGYPVYV
2,104,CAAMEGAQKLVF,TRAV13-1*01,TRAJ54*01,CASSYPGGGFYEQYF,TRBV6-5*01,TRBJ2-7*01,HomoSapiens,HLA-A*02:01:48,MHCI,LLFGYPVYV
3,114,CAGAGSQGNLIF,TRAV27*01,TRAJ42*01,CASSSRSSYEQYF,TRBV19*01,TRBJ2-7*01,HomoSapiens,HLA-A*02:01:48,MHCI,GILGFVFTL
4,115,CAVTTDSWGKLQF,TRAV12-2*01,TRAJ24*01,CASRPGLAGGRPEQYF,TRBV6-5*01,TRBJ2-7*01,HomoSapiens,HLA-A*02:01:59,MHCI,LLFGYAVYV


In [43]:
# vdjdb = vdjdb.drop('complex.id', axis=1)
# print('unique TCR-pMHC complex:', vdjdb.shape)
# complex = []
# for i in range(len(vdjdb)):
#     complex.append((vdjdb['cdr3a'].iloc[i], vdjdb['cdr3b'].iloc[i], vdjdb['epitope'].iloc[i]))
# vdjdb['complex'] = complex
# vdjdb_dict = vdjdb.drop_duplicates(subset='complex').set_index('complex', drop=True).to_dict(orient='index')
# print('unique tcr-peptide pairs:', len(vdjdb_dict.keys()))

In [51]:
# final data formating
try:
    vdjdb = vdjdb.drop('complex.id', axis=1)
except:
    pass
vdjdb = vdjdb.replace('MHCI', 1) # name formatting
vdjdb = vdjdb.replace('HomoSapiens', 'human') # name formatting
vdjdb = vdjdb[vdjdb['species']=='human'].copy() # keep only humans
vdjdb = vdjdb.drop_duplicates()
print("preprocessed vdjdb:", vdjdb.shape)
vdjdb.head()

preprocessed vdjdb: (967, 10)


,cdr3a,va,ja,cdr3b,vb,jb,species,mhc,mhc_class,epitope
0,CAVAGYGGSQGNLIF,TRAV12-2*01,TRAJ42*01,CASSPQGLGTEAFF,TRBV28*01,TRBJ1-1*01,human,HLA-A*02,1,ELAGIGILTV
1,CAVTTDSWGKLQF,TRAV12-2*01,TRAJ24*01,CASRPGLAGGRPEQYF,TRBV6-5*01,TRBJ2-7*01,human,HLA-A*02:01:59,1,LLFGYPVYV
2,CAAMEGAQKLVF,TRAV13-1*01,TRAJ54*01,CASSYPGGGFYEQYF,TRBV6-5*01,TRBJ2-7*01,human,HLA-A*02:01:48,1,LLFGYPVYV
3,CAGAGSQGNLIF,TRAV27*01,TRAJ42*01,CASSSRSSYEQYF,TRBV19*01,TRBJ2-7*01,human,HLA-A*02:01:48,1,GILGFVFTL
4,CAVTTDSWGKLQF,TRAV12-2*01,TRAJ24*01,CASRPGLAGGRPEQYF,TRBV6-5*01,TRBJ2-7*01,human,HLA-A*02:01:59,1,LLFGYAVYV


In [53]:
vdjdb.columns

Index(['cdr3a', 'va', 'ja', 'cdr3b', 'vb', 'jb', 'species', 'mhc', 'mhc_class',
       'epitope'],
      dtype='object')

## IEDB

IEDB queried for MHC 1 and human only

In [58]:
subset = ['Chain 1 CDR3 Curated', 'Chain 2 CDR3 Curated']
iedb = iedb_raw.dropna(subset=subset)
iedb = iedb.astype('str')
print("Full IEDB:", iedb.shape)

iedb_base_cols = ['Group Receptor ID', 'Receptor ID', 'Description', 'Organism', 'MHC Allele Names']

iedb_cols = [col for col in iedb.columns if "Curated" in col and ("CDR3" in col or "V" in col or "J" in col)]
iedb_cols = ['Description', 'Organism', 'MHC Allele Names', 'Curated Chain 1 V Gene', 'Curated Chain 1 J Gene', 'Chain 1 CDR3 Curated',  
            'Curated Chain 2 V Gene','Curated Chain 2 J Gene', 'Chain 2 CDR3 Curated']
iedb = iedb[iedb_cols]
print("IEDB Selected Columns:", iedb.shape)

iedb = iedb.replace('nan', np.nan)
iedb = iedb.dropna()
iedb.columns = ['epitope', 'species', 'mhc', 'va', 'ja', 'cdr3a', 'vb', 'jb', 'cdr3b']
iedb['mhc_class'] = 1
iedb = iedb[vdjdb.columns]
print('IEDB remove nan:', iedb.shape)
iedb = iedb.drop_duplicates()
print('IEDB remove duplicates:', iedb.shape)

# we need the CDR3 ends for 3D generation, one way to get this is to check if the sequence starts with C
iedb = iedb[iedb['cdr3a'].str.startswith('C')]
iedb = iedb[iedb['cdr3b'].str.startswith('C')]
print('IEDB with cdr3 ends only:', iedb.shape)
iedb

Full IEDB: (26710, 71)
IEDB Selected Columns: (26710, 9)
IEDB remove nan: (8492, 10)
IEDB remove duplicates: (8164, 10)
IEDB with cdr3 ends only: (7407, 10)


,cdr3a,va,ja,cdr3b,vb,jb,species,mhc,mhc_class,epitope
93,CAVRPTSGGSYIPTF,TRAV21*01,TRAJ6*01,CASSYVGNTGELFF,TRBV6-5*01,TRBJ2-2*01,Homo sapiens (human),HLA-A*02:01,1,SLLMWITQC
187,CAGGTGNQFYF,TRAV35*02,TRAJ49*01,CAISEVGVGQPQHF,TRBV10-3,TRBJ1-5*01,Homo sapiens (human),HLA-A*02:01,1,AAGIGILTV
712,CAASETSGSWQLIFG,TCRAV14-2,TCRAJ22,CASSRDLGRDTYQYFG,TCRBV17,TCRBJ2-5,Influenza A virus,H2-Db,1,ASNENMETM
713,CAASETSGSWQLIFG,TCRAV14-2,TCRAJ22,CASSRDLGRDTYQYFG,TCRBV17,TCRBJ2-5,Influenza A virus,H2-Db,1,ASNENMEAM
715,CAASETSGSWQLIFG,TCRAV14-2,TCRAJ22,CASSRDLGRDTYQYFG,TCRBV17,TCRBJ2-5,H1N1 subtype (H1N1),H2-Db,1,ASNENAETM
...,...,...,...,...,...,...,...,...,...,...
27059,CGAGETSGSRLTF,TRAV21,TRAJ58,CSVNLGGPTDTQYF,TRBV29-1,TRBJ2-3,SARS-CoV2,HLA-B*07:02,1,KPVETSNSF
27060,CALEGSQGNLIF,TRAV9-2,TRAJ42,CSVPDGAEPYGYTF,TRBV20-1,TRBJ1-2,SARS-CoV2,HLA-A*01:01,1,TTDPSFLGRY
27061,CLVGNTGGFKTIF,TRAV4,TRAJ9,CSVPDRGNTEAFF,TRBV29-1,TRBJ1-1,SARS-CoV2,HLA-B*07:02,1,SPRWYFYYL
27063,CAPSRHAGNNRKLIW,TRAV9-2,TRAJ38,CSVQGGTNEKLFF,TRBV29-1,TRBJ1-4,Influenza A virus,HLA-A*01:01,1,VSDGGPNLY


## McPAS-TCR

In [78]:
mcpas_cols = ['CDR3.alpha.aa', 'CDR3.beta.aa', 'Species', 'Epitope.peptide', 'MHC', 'TRAV', 'TRAJ', 'TRBV', 'TRBJ']
mcpas = mcpas_raw[mcpas_cols].copy()
print('McPAS-TCR raw:', mcpas.shape)
mcpas = mcpas.dropna()
print('McPAS-TCR drop na:', mcpas.shape)
mcpas = mcpas.drop_duplicates()
print('McPAS-TCR drop na:', mcpas.shape)

# formatting
mcpas['Species'] = mcpas['Species'].str.lower()
mcpas.columns = ['cdr3a', 'cdr3b', 'species', 'epitope', 'mhc', 'va', 'ja', 'vb', 'jb']
mcpas['mhc_class'] = 1
mcpas = mcpas[vdjdb.columns].copy()
mcpas

McPAS-TCR raw: (39985, 9)
McPAS-TCR drop na: (3124, 9)
McPAS-TCR drop na: (2914, 9)


,cdr3a,va,ja,cdr3b,vb,jb,species,mhc,mhc_class,epitope
73,CALGLMSNYNVLYF,TRAV4,TRAJ14-4,CASSSGLGGTLYF,TRBV10,TRBJ2-4,mouse,H-2Db,1,SSGVENPGGYCLTKW
80,CAAETTASLGKLQF,TRAV11,TRAJ9,CASGDHGLSYEQYF,TRBV13-3,TRBJ2-6,mouse,H-2b,1,DEPLTSLTPRCNTAWNRLKL
84,CALGDRGSGGSNYK,TRAV4,TRAJ3DT,CAWSRTGGNSDYTF,TRBV31,TRBJ1-2,mouse,H-2b,1,DEPLTSLTPRCNTAWNRLKL
120,CAAEASSSFSKLVF,TRAV11-1,TRAJ42,CASAPDRGGERLF,TRBV8-2,TRBJ1-4,mouse,H-2q,1,GPEGAQGPRGEPGTP
121,CAAEASSSFSKLVF,TRAV11-1,TRAJ42,CASAPDRGGERLF,TRBV13-2,TRBJ1-4,mouse,H-2q,1,GPEGAQGPRGEPGTP
...,...,...,...,...,...,...,...,...,...,...
39028,CALDGPSNTGKLIF,TRAV16,TRAJ37,CATSESSGQTYEQYF,TRBV15,TRBJ2-2,human,HLA-A2:01,1,FLCMKALLL
39029,CATDAEGNNRLAF,TRAV17,TRAJ7,CASSIFGGGLGEQFF,TRBV19,TRBJ2-7,human,HLA-A2:01,1,FLCMKALLL
39030,CGAVGYQKVTF,TRAV34,TRAJ13,CALNGEISYNEQFF,TRBV2,TRBJ2-2,human,HLA-A2:01,1,FLCMKALLL
39031,CAVIWYNNNDMRF,TRAV8-1,TRAJ43,CASSQGVNTGELFF,TRBV4-2,TRBJ2-1,human,HLA-A2:01,1,FLCMKALLL


In [80]:
# keep only HLA
mcpas = mcpas[mcpas['mhc'].str.startswith('HLA-A')]
mcpas

,cdr3a,va,ja,cdr3b,vb,jb,species,mhc,mhc_class,epitope
152,CVVKKNNTDKLIF,TRAV2,TRAJ2,CASSQGTSQFNEQF,TRBV7,TRBJ2-1,human,HLA-A*02,1,EAAGIGILTV
153,CAVGLGGANNLFF,TRAV9,TRAJ5,CASSSGQGLNIQYF,TRBV4,TRBJ2-4,human,HLA-A*02,1,EAAGIGILTV
154,CAVNFAGGGADGLTF,TRAV1-1,TRAJ45,CASSESAGGYYNEQF,TRBV2,TRBJ2-1,human,HLA-A*011,1,IVTDFSVIK
155,CAGHAGNQFYF,TRAV25,TRAJ49,CASSFGFGSSYGYTF,TRBV11-01,TRBJ1-2,human,HLA-A*011,1,IVTDFSVIK
157,CALIQGAQKLVF,TRAV9,TRAJ54,CASSLEPGRNEKLF,TRBV7-08,TRBJ1-4,human,HLA-A*011,1,IVTDFSVIK
...,...,...,...,...,...,...,...,...,...,...
39028,CALDGPSNTGKLIF,TRAV16,TRAJ37,CATSESSGQTYEQYF,TRBV15,TRBJ2-2,human,HLA-A2:01,1,FLCMKALLL
39029,CATDAEGNNRLAF,TRAV17,TRAJ7,CASSIFGGGLGEQFF,TRBV19,TRBJ2-7,human,HLA-A2:01,1,FLCMKALLL
39030,CGAVGYQKVTF,TRAV34,TRAJ13,CALNGEISYNEQFF,TRBV2,TRBJ2-2,human,HLA-A2:01,1,FLCMKALLL
39031,CAVIWYNNNDMRF,TRAV8-1,TRAJ43,CASSQGVNTGELFF,TRBV4-2,TRBJ2-1,human,HLA-A2:01,1,FLCMKALLL


## Combine

In [90]:
positives = pd.concat((iedb, vdjdb, mcpas), axis=0)
print("concate db:", positives.shape)
positives = positives.drop_duplicates(subset=['cdr3a', 'cdr3b', 'epitope'], keep='last')
print("concate db remove duplicates:", positives.shape)

concate db: (9383, 10)
concate db remove duplicates: (8381, 10)


In [93]:
complex = []
for i in range(len(positives)):
    complex.append((positives['cdr3a'].iloc[i], positives['cdr3b'].iloc[i], positives['epitope'].iloc[i]))
positives['complex'] = complex
positives_dict = positives.drop_duplicates(subset='complex').set_index('complex', drop=True).to_dict(orient='index')
try:
    positives = positives.drop('complex', axis=1)
except:
    pass
print('unique tcr-peptide pairs:', len(positives_dict.keys()))

unique tcr-peptide pairs: 8381


# Negative sample generation

In [95]:
i = 0
negatives = []
while i<len(positives):
    sample_idx = random.choice([k for k in range(len(positives)) if k!=i])
    neg_complex = (positives['cdr3a'].iloc[sample_idx], positives['cdr3b'].iloc[sample_idx], positives['epitope'].iloc[i])
    try:
        positives_dict[neg_complex]
    except:
        sample = positives.iloc[sample_idx].copy()
        sample['epitope'] =  positives['epitope'].iloc[i]
        negatives.append(sample.values)
    i+=1
negatives = pd.DataFrame(negatives, columns=positives.columns)
